# Продвинутое машинное обучение: 
## Домашнее задание 3
Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования. Как и раньше, в качестве решения ожидается ссылка на jupyter-ноутбук на вашем github (или публичный, или с доступом для snikolenko); ссылку обязательно нужно прислать в виде сданного домашнего задания на портале Академии. Как всегда, любые комментарии, новые идеи и рассуждения на тему категорически приветствуются. 

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса.

Расшифруем несколько предложений простым частотным алгоритмом.

In [20]:
import re
from collections import Counter
import random
import math

In [3]:
with open('corpora/WarAndPeace.txt', 'r') as f:
    text_ru = f.read()
    
text_ru[:1000], len(text_ru)

('Annotation\n\n\n\t"Война и мир" – самый известный роман Льва Николаевича Толстого, как никакое другое произведение писателя, отражает глубину его мироощущения и философии.\tЭта книга из разряда вечных, потому что она обо всем – о жизни и смерти, о любви и чести, о мужестве и героизме, о славе и подвиге, о войне и мире.\tПервый том знакомит с высшим обществом России XIX века. Показаны взаимоотношения между родителями и детьми в семье Ростовых, сватовство у Болконских, интриги у Безуховых, вечера в салоне фрейлины А.П.Шерер, балы в Москве и Петербурге.\n\n\n\n\n\n* * *\n\n\n\nЛев Николаевич ТолстойЧАСТЬ ПЕРВАЯ\n\nЧАСТЬ BTОРАЯ\n\nЧАСТЬ ТРЕТЬЯ\n\n\n\n\n\n* * *\n\n\n\n\n\nЛев Николаевич Толстой\n\nВОЙНА И МИР\n\nТом 1\n\n\n\n\nЧАСТЬ ПЕРВАЯ\n\n\n\n\n\nI\n\n\n— Еh bien, mon prince. Gênes et Lucques ne sont plus que des apanages, des поместья, de la famille Buonaparte. Non, je vous préviens, que si vous ne me dites pas, que nous avons la guerre, si vous vous permettez encore de pallier toute

In [4]:
with open('corpora/WarAndPeaceEng.txt', 'r') as f:
    text_eng = f.read()
    
text_eng[:1000]

'\ufeffThe Project Gutenberg EBook of War and Peace, by Leo Tolstoy\n\nThis eBook is for the use of anyone anywhere at no cost and with almost\nno restrictions whatsoever.  You may copy it, give it away or re-use it\nunder the terms of the Project Gutenberg License included with this\neBook or online at www.gutenberg.org\n\n\nTitle: War and Peace\n\nAuthor: Leo Tolstoy\n\nTranslators: Louise and Aylmer Maude\n\nPosting Date: January 10, 2009 [EBook #2600]\n\nLast Updated: March 15, 2013\n\nLanguage: English\n\n\n*** START OF THIS PROJECT GUTENBERG EBOOK WAR AND PEACE ***\n\nAn Anonymous Volunteer, and David Widger\n\n\n\n\n\nWAR AND PEACE\n\nBy Leo Tolstoy/Tolstoi\n\nCONTENTS\n\nBOOK ONE: 1805\n\nCHAPTER I\n\nCHAPTER II\n\nCHAPTER III\n\nCHAPTER IV\n\nCHAPTER V\n\nCHAPTER VI\n\nCHAPTER VII\n\nCHAPTER VIII\n\nCHAPTER IX\n\nCHAPTER X\n\nCHAPTER XI\n\nCHAPTER XII\n\nCHAPTER XIII\n\nCHAPTER XIV\n\nCHAPTER XV\n\nCHAPTER XVI\n\nCHAPTER XVII\n\nCHAPTER XVIII\n\nCHAPTER XIX\n\nCHAPTER XX\n\nCH

In [5]:
def simple_text_preparation(text, regex):
    text = text.lower()
    text = text.replace('.', ' ')
    text = text.strip()
    text = re.sub(regex, '', text)
    return text

text_ru = simple_text_preparation(text_ru, r'[^а-я ]')
text_end = simple_text_preparation(text_eng, r'[^a-z ]')

counter_ru = Counter(text_ru)
counter_ru.most_common(10)

[(' ', 117256),
 ('о', 61282),
 ('а', 45209),
 ('е', 42519),
 ('и', 35838),
 ('н', 35119),
 ('т', 30619),
 ('с', 28128),
 ('л', 27277),
 ('в', 24824)]

In [63]:
def list_shuffle(l):
    l.remove(' ')
    before_ = l.copy()
    random.shuffle(l)
    after_ = l
    return dict(zip(before_, after_))


def text_shuffle(text, shuffle):
    new_text = list(text)
    for i, letter in enumerate(text):
        if letter in shuffle.keys():
            new_text[i] = shuffle[letter]
    return ''.join(new_text)
    
shuffle = list_shuffle(list(counter_ru))

new_text = text_shuffle(text_ru, shuffle)
    
new_text[:10], text_ru[:10]

('йкюьр е пе', 'война и ми')

Так мы производим замену букв в тексте с помощью случайной пересатновки алфавита. Попробуем восстановить текст по такой расстановке.

In [64]:
sample_size = 1500

train_begin = random.randint(0, len(text_ru) - sample_size)
test_begin = random.randint(0, len(text_ru) - sample_size)
train_text = text_ru[train_begin: train_begin + sample_size]
test_text = text_ru[test_begin: test_begin + sample_size]

train_text, test_text

('розного начальства во всем ужасе представилась его вина и позор в том что он оставшись жив потерял два орудия  он так был взволнован что до сей минуты не успел подумать об этом  смех офицеров еще больше сбил его с толку  он стоял перед багратионом с дрожащею нижнею челюстью и едва проговорил не знаю ваше сиятельство людей не было ваше сиятельство  вы бы могли из прикрытия взятьчто прикрытия не было этого не сказал тушин хотя это была сущая правда  он боялся подвести этим другого начальника и молча остановившимися глазами смотрел прямо в лицо багратиону как смотрит сбившийся ученик в глаза экзаменатору молчание было довольно продолжительно  князь багратион видимо не желая быть строгим не находился что сказать остальные не смели вмешаться в разговор  князь андрей исподлобья смотрел на тушина и пальцы его рук нервически двигались  ваше сиятельство  прервал князь андрей молчание своим резким голосом  вы меня изволили послать к батарее капитана тушина  я был там и нашел две трети людей и 

In [65]:
counter_train = Counter(train_text)
shuffle = list_shuffle(list(counter_train))

train_text_shuffled = text_shuffle(train_text, shuffle)
test_text_shuffled = text_shuffle(test_text, shuffle)
train_text_shuffled, test_text_shuffled

('юшкчшош чуфумибьлу лш лбвт ждубв аювзбьулемуби вош лечу е ашкшю л ьшт фьш шч шбьулсеби дел ашьвюэм злу шюжзеэ  шч ьущ цым лклшмчшлуч фьш зш бвп течжьы чв жбавм ашзжтуьи шц гьшт  бтвр шйенвюшл вяв цшмисв бцем вош б ьшмщж  шч бьшэм авювз цуоюуьешчшт б зюшдуявх чедчвх фвмхбьих е взлу аюшошлшюем чв кчух лусв беэьвмибьлш мхзвп чв цымш лусв беэьвмибьлш  лы цы тшоме ек аюещюыьеэ лкэьифьш аюещюыьеэ чв цымш гьшош чв бщукум ьжсеч ршьэ гьш цыму бжяуэ аюулзу  шч цшэмбэ ашзлвбье гьет зюжошош чуфумичещу е тшмфу шбьучшлелсетебэ омукуте бтшьювм аюэтш л менш цуоюуьешчж щущ бтшьюеь бцелсепбэ жфвчещ л омуку гщкутвчуьшюж тшмфучев цымш зшлшмичш аюшзшмдеьвмичш  щчэки цуоюуьешч лезетш чв двмуэ цыьи бьюшоет чв чуршзембэ фьш бщукуьи шбьумичыв чв бтвме лтвсуьибэ л юукошлшю  щчэки учзювп ебашзмшциэ бтшьювм чу ьжсечу е аумины вош южщ чвюлефвбще злеоумеби  лусв беэьвмибьлш  аювюлум щчэки учзювп тшмфучев блшет ювкщет ошмшбшт  лы твчэ еклшмеме ашбмуьи щ цуьуювв щуаеьучу ьжсечу  э цым ьут е чусвм злв ьювье мхзвп е 

In [66]:
counter_decode = Counter(test_text_shuffled)

encoded_freq_order = [letter[0] for letter in sorted(counter_decode.items(), key=lambda x: x[1], reverse=True)]
initial_freq_order = [letter[0] for letter in sorted(counter_ru.items(), key=lambda x: x[1], reverse=True)]

# make encoding
for i in range(max(len(encoded_freq_order) - len(initial_freq_order), 0)):
    initial_freq_order.append('*')
    
decoder = dict(zip(encoded_freq_order, initial_freq_order))

decoded_text = list(test_text_shuffled)
for i, letter in enumerate(decoded_text):
    decoded_text[i] = decoder[letter]

''.join(decoded_text)

' ойимаепге жауи рткыь вчомин бгслдо весено рар от и всеума чомин рар бзмло зпгжнетто своипи лодоянивгпи патедапи ядемславнкк ядоливояонойтосль слдоуопз яодкмрз мопа в ло йе путоветие боньжие шасг ядобини мва и лотрип уоносроп олоыванись в уослитоц мдзуие  рткыь ослатовинск иыяом вискшич узслгч бдовец ойивнеттге бнеслкюие слдоуие унаыа оункмени всеч и ослатовинись та пономоц рткуите  пономак рткуитк исяглгвана в ло вдепк ло шзвслво рарое исяглгвахл ядимводтге та эадсроп вгчоме ло шзвслво слдача и яошлетик ролодое воыбзйман щлол сладир во всеч ядибнийеттгч  от яоунамин рткуитх яо уонове и яолоп теноврип мвийетиеп яолдеяан ее яо ыалгнрз  к дам к дам  ядоуоводин от и ядисланьто еюе выунктзв ец в унаыа бгслдо оложен и сен та свое песло   самилесь самилесь пичаин иватовиш самилесь от зраыан тевеслре песло яомне себк  офиэиатл оломвитзн мнк тее слзн  уо уо  сраыан сладир оункмгвак ее ордзунеттзх ланих   яолодояинась течодожоот ыаспекнск сзчо чономто теядиклто рар от всеума спекнск омтип длоп

 Получается, конечно, не очень. Можно лучше

## Проделаем то же самое для биграмм

In [67]:
bigrams = []
list_text_ru = list(text_ru)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams.append(a + b)
        

In [68]:
bigrams_counter_ru = Counter(bigrams)

bigrams_encoded = []
list_text_ru = list(test_text_shuffled)
for a, b in zip(list_text_ru[:-1], list_text_ru[1:]):
    if a != ' ' and b != ' ':
        bigrams_encoded.append(a + b)
    
bigrams_encoded_counter = Counter(bigrams_encoded)

encoded_freq_order = [letter[0] for letter in sorted(bigrams_encoded_counter.items(), key=lambda x: x[1], reverse=True)]
initial_freq_order = [letter[0] for letter in sorted(bigrams_counter_ru.items(), key=lambda x: x[1], reverse=True)]
    
decoder = dict(zip(encoded_freq_order, initial_freq_order))

decoded_text = list(test_text_shuffled)
for i, letter in enumerate(decoded_text[:-1]):
    if decoded_text[i] != ' ' and decoded_text[i+1] != ' ':
        decoded_text[i] = decoder[decoded_text[i] + decoded_text[i+1]][0]

''.join(decoded_text)

' дииожисев тяое ентки ьпкиам кзтенш етоблш нвщ рч е аоиыоу пкиам нвщ узегш цсыылзнош акемие глнхбсмырие мсоддрие овшутоиэмиэ онесмкхрвлдчовтии тенлодаж рлятеуж одму л гш дв геооезрав евмбжав шеьы онаеасе ату е грншмт овлваодт еглутпсчеи л овтсрогп увжоав  ентки втосооааусэ усркз ачсаатр хатвнр еноееп димэлзннев олттевоав тенлоав тсрну лтмтубсе аокр е втосооаасчеи оу свлкогп енжоров  свлкоиэ енжорнэ чввувктпсу л гш ьвидэ гш ллатбкш нвосв чввувктеыь оаиаклтнев оу оияаодт ынпкув гш ллатбкш тедьсу е рягтзраэ оеглнсв клуущаопм ыгеь тоиалщ кш аокр оауосидзнннр  рч рлтслиам енжорлх рш овлоев е регдт облогшмт ааидзраит реевчтпм ав рш нсврвуж  э длз э длз  онлооклаам рч е оачтопмоош пов лгтмаплл еп л тсрну кзтенш егбибм е обм оу аксв чттгш   илинттеи илинттеи итсыам мтсооааф илинттеи рч бнрнпм оуеттзпв чттгш ркблв осбэ  оааессяь егкаарпем бмэ оав тпем  ош ош  анрнпм тоиалщ лтмтсктиэ ав нувжтлзнпзх опспх   реглнхиасееи окплнбазрч неичмсусэ дйпш пвлкдош очоаащоош нвщ рч аоиыоу ичмсусэ кдрмт егдт

# MCMC

Для каждого шифра (правила замены символа) однозначно определяется ключ. Наша задача найти ключ, имея информацию о том, с какой частотой распределены биграммы в языке. Мы будем сюмплировать ключи и двигаясь в направлении наибольшего правдоподобия расшифрованного текста, сойдемся к стационарному распределению. В качестве меры качества расшифровки будем считать 

$score = \sum_{i=1}^{n} f_i \log{p_i},$

где $p_i$ - частота i-ой биграммы в Войне и Мире, а $f_i$ - в расшифрованном тесте. Чем точнее расшифровываем, тем выше score. 

In [69]:
# calculate score

def decoding_score(text):
    decoded_freq = {}
    list_text = list(text)
    for a, b in zip(list_text[:-1], list_text[1:]):
        if a != ' ' and b != ' ':  # count occurence of every bigram in the text
            bigram = a + b
            if bigram in decoded_freq:
                decoded_freq[bigram] += 1
            else:
                decoded_freq[bigram] = 1
    score = 0
    for k, v in decoded_freq.items():
        if k in bigrams_counter_ru:
            score += v * math.log(bigrams_counter_ru[k]) 
        
    return score

t = "дви оэ къисзжу к сжэюпои поунжхг лжсэолоаэчр коймб нломплжэмпко эжроаюеиимю сж аиликэит нлжшох  япо пжбои япо пжбои но бох мплиуютп бпо мплиуюип  мнлжвгкжу ломпок локэююмд м лцммбгхг г жкмплгймбгхг моуажпжхг ыизжквгхг нилихивжээчхг поунжхг эжнилилис иьо аолоьг  ж яолп гр сэжип кмир ноыгу нлонжажй км  опкияжуг ихц нолцммбг ноэихишбг г нояивмбг поунч ыиьцегр г эиноэгхжквгр пояэо пжб зи бжб г оэ поьо япо пцп аиужуомд  ыий эихшик  блгяжу оагэ  ж яолп гр аилг  гсхиээгбок      б яолпц щпгр лцммбгр  япопо коляжу эихиш эимбоудбо лжэиэчр вуг но аолоьи  лцьжпиудмпкж блгбг мпоэч мугкжугмд к оагэ оыегй ьцу  мплиудыж сжпгруж г бжб нопох цсэжу ломпок мплиуюуг алць к алцьж лцммбги г жкмплгймбги моуажпч ыози хой япо з щпо пжбои  ацхжу ломпок   г саимд ьаи кмюбцт хгэцпц ьомцажлд хозип цкгажпд гр эо эип щпо килэо поудбо эимбоудбо хилсжкшик  щпо нлойаип щпо эи по щпо эи хозип ычпд  ацхжу оэ   поудбо номболии номболии нлоиржпд грхчмуд о нолжзиэгг г ыиьмпки эи хоьуж нлгппг к ьоуокц ломпокц  ропю оэ г кгаиу"
decoding_score(t)

2639.108615172398

В качестве ключа будем использовать последовательность букв, которые и будут задавать соответствие расшифровки

In [70]:
def swap_letters(sh):
    """Это функция, которая меняет в ключе два символа между собой."""
    shuffle = sh.copy()
    pos_1, pos_2 = random.choices(list(shuffle.keys()), k=2)
    if pos_1 == pos_2:
        return swap_letters(shuffle)
    value_1 = shuffle[pos_1]
    value_2 = shuffle[pos_2]
    shuffle[pos_2] = value_1
    shuffle[pos_1]= value_2
    return shuffle


In [71]:
def MCMC_decrypt(n_iter, encoded_text, sh):
    shuffle = sh.copy()
    best_key = {}
    score = 0
    for i in range(n_iter):
        # изменим две буквы в текущем ключе
        new_shuffle = swap_letters(shuffle)
        
        # расшифруем закодированный текст с помощью двух вариантов ключа
        new_decoded_text = text_shuffle(encoded_text, new_shuffle)
        decoded_text = text_shuffle(encoded_text, shuffle)
        
        # посчитаем score для двух вариантов ключей
        current_score = decoding_score(decoded_text)
        new_score = decoding_score(new_decoded_text)
        
        acceptance_probability = min(1, math.exp(new_score - current_score))
        if current_score > score:
            best_key = shuffle
            score = current_score
        if random.uniform(0, 1) < acceptance_probability:
            shuffle = new_shuffle
        if i%500==0:
            print(f"{i=} {decoded_text[0:99]}")
    return best_key, decoded_text


random_shuffle = list_shuffle(list(counter_ru))
MCMC_decrypt(10000, train_text_shuffled, random_shuffle)


i=0 элрвлол вфбфенгщуф ул угпк счфгп тэпдгщфуъефгн пол уъвф ъ тлрлэ у щлк бщл лв лгщфуйъгн чъу тлщпэзе 
i=500 реждече дацатнивма ме миуш схаиу фрупивамотаин уче мода о фежер м веш цве ед еивамкоин хом февурлт 
i=1000 рездече даьалнутва ве вуим сфауи грипутаволаун иче вода о гезер в тем ьте ед еутавкоун фов гетирял 
i=1500 резнече насалудтва ве вдим ьшади пригдтаволаду иче вона о пезер в тем сте ен едтавкоду шов петирял 
i=2000 рознодо начальства во всем угасе прекставилась едо вина и позор в том что он оставшись гив потерял 
i=2500 розного нажальства во всем учасе прекставилась его вина и позор в том жто он оставшись чив потерял 
i=3000 розного начальства во всем ужасе представилась его вина и позор в том что он оставшись жив потерял 
i=3500 розного начальства во всем ужасе представилась его вина и позор в том что он оставшись жив потерял 
i=4000 розного начальства во всем ужасе представилась его вина и позор в том что он оставшись жив потерял 
i=4500 розного начальства во всем ужасе п

({'в': 'е',
  'о': 'г',
  'й': 'ф',
  'н': 'ц',
  'а': 'п',
  'и': 'ь',
  'м': 'л',
  'р': 'х',
  'с': 'ш',
  'ы': 'ы',
  'з': 'д',
  'е': 'и',
  'т': 'м',
  'л': 'в',
  'ь': 'т',
  'к': 'з',
  'ч': 'н',
  'г': 'э',
  'д': 'ж',
  'у': 'а',
  'п': 'й',
  'я': 'щ',
  'ж': 'у',
  'б': 'с',
  'щ': 'к',
  'ф': 'ч',
  'э': 'я',
  'х': 'ю',
  'ю': 'р',
  'ш': 'о',
  'ц': 'б',
  'ъ': 'ъ'},
 'розного начальства во всем ужасе представилась его вина и позор в том что он оставшись жив потерял два орудия  он так был взволнован что до сей минуты не успел подумать об этом  смех офицеров еще больше сбил его с толку  он стоял перед багратионом с дрожащею нижнею челюстью и едва проговорил не знаю ваше сиятельство людей не было ваше сиятельство  вы бы могли из прикрытия взятьчто прикрытия не было этого не сказал тушин хотя это была сущая правда  он боялся подвести этим другого начальника и молча остановившимися глазами смотрел прямо в лицо багратиону как смотрит сбившийся ученик в глаза экзаменатору молч

Таким образом алгоритм MCMC сошелся за 1000 итераций и полностью расшифровал текст.